# 🔧 문장 순서 예측 전처리 노트북
- Pairwise 방식용 CSV 생성
- Seq2Seq 방식용 CSV 생성
- 저장 경로: `data/cleaned_pairwise.csv`, `data/cleaned_seq2seq.csv`

In [1]:
import pandas as pd
import itertools
import os

## ✅ 데이터 불러오기

In [ ]:
# 원본 데이터 로드
df = pd.read_csv("../data/train.csv")
df.head()

,ID,sentence_0,sentence_1,sentence_2,sentence_3,answer_0,answer_1,answer_2,answer_3
0,TRAIN_0000,블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다.,"이러한 특성은 유권자들에게 신뢰를 제공하며, 민주적 참여를 촉진하는 데 기여할 수 있다.",결과적으로 블록체인 기반의 투표 시스템은 공정하고 신뢰할 수 있는 선거 환경을 조성...,각 투표는 변경 불가능한 기록으로 저장되어 조작의 가능성을 원천적으로 차단한다.,0,3,1,2
1,TRAIN_0001,줄거리 자동 생성의 인공지능 알고리즘은 대량의 텍스트 데이터를 분석하여 핵심 정보를...,"결과적으로, 이러한 기술은 사용자에게 신속하고 효율적인 정보 전달을 가능하게 한다.",생성된 줄거리는 원본 텍스트의 의미를 유지하면서도 간결하게 요약된 형태로 제공된다.,"이 알고리즘은 자연어 처리 기술을 활용하여 문맥을 이해하고, 주요 사건과 등장인물을...",0,3,2,1
2,TRAIN_0002,"마지막으로, 키친타올을 보관할 때는 쉽게 접근할 수 있는 곳에 두어 낭비를 방지하는...",재사용 가능한 천이나 스펀지를 활용하면 키친타올의 필요성을 줄일 수 있다.,물기를 제거할 때는 가볍게 눌러주어 과도한 사용을 피할 수 있다.,키친타올을 절약하는 첫걸음은 필요한 양만큼만 사용하는 것이다.,3,2,1,0
3,TRAIN_0003,책의 페이지가 손상되지 않도록 수직으로 세워 두거나 평평하게 눕혀 보관하는 것이 좋다.,"정기적으로 먼지를 털어내고, 곰팡이나 해충의 발생 여부를 점검하는 것이 중요하다.",종이책은 직사광선이 닿지 않는 서늘하고 건조한 장소에 보관해야 한다.,"필요할 경우, 책을 보호하기 위해 커버를 씌우거나 전용 보관함에 넣는 방법도 고려할...",2,0,1,3
4,TRAIN_0004,"인공지능 모델은 반복적인 실험을 통해 지속적으로 학습하며, 이를 통해 발견의 정확성...",인공지능은 대량의 데이터를 분석하여 숨겨진 패턴과 상관관계를 발견하는 데 강력한 도...,"결국, 인공지능의 지원은 과학적 발견의 속도와 효율성을 혁신적으로 변화시킬 수 있는...",이러한 분석 결과는 연구자들에게 새로운 가설을 제시하고 실험 설계를 개선하는 데 기...,1,3,0,2


## ✅ Pairwise 전처리 함수

In [3]:
def preprocess_pairwise(df):
    data = []
    for _, row in df.iterrows():
        sentences = [row[f'sentence_{i}'] for i in range(4)]
        answer = [row[f'answer_{i}'] for i in range(4)]
        ordered = [sentences[i] for i in answer]
        positive_pairs = [(ordered[i], ordered[i+1]) for i in range(3)]
        all_pairs = list(itertools.permutations(sentences, 2))
        for s1, s2 in all_pairs:
            label = 1 if (s1, s2) in positive_pairs else 0
            data.append({'sentence1': s1, 'sentence2': s2, 'label': label})
    return pd.DataFrame(data)

## ✅ Seq2Seq 전처리 함수

In [4]:
def preprocess_seq2seq(df):
    data = []
    for _, row in df.iterrows():
        sentences = [row[f'sentence_{i}'] for i in range(4)]
        answers = [row[f'answer_{i}'] for i in range(4)]
        input_text = ' [SEP] '.join(sentences)
        target_text = ' '.join(map(str, answers))
        data.append({'input_text': input_text, 'target_text': target_text})
    return pd.DataFrame(data)

## ✅ 전처리 실행 및 저장

In [11]:
# Pairwise 저장
pairwise_df = preprocess_pairwise(df)
os.makedirs("../data", exist_ok=True)
pairwise_df.to_csv("../data/cleaned_pairwise.csv", index=False)
print("[✅] cleaned_pairwise.csv 저장 완료")

# Seq2Seq 저장
seq2seq_df = preprocess_seq2seq(df)
seq2seq_df.to_csv("../data/cleaned_seq2seq.csv", index=False)
print("[✅] cleaned_seq2seq.csv 저장 완료")

[✅] cleaned_pairwise.csv 저장 완료
[✅] cleaned_seq2seq.csv 저장 완료
